In [1]:
import os
import numpy as np
from keras.models import Sequential # To initialise the nn as a sequence of layers
from keras.layers import Convolution2D # To make the convolution layer for 2D images
from keras.layers import MaxPooling2D # 
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import CSVLogger
from tensorflow.keras.optimizers import RMSprop
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

csv=CSVLogger("1_rmsprop100ep.log")
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')


# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32,(2,2),input_shape = (224,224,1), activation = 'relu',strides=2,name='convo1'))
classifier.add(Convolution2D(64,(3,3), activation = 'relu',name='convo2'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Step 1 - Convolution
classifier.add(Convolution2D(64,(3,3),activation = 'relu',name='convo3'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))
# Step 3 - Flattening
classifier.add(Convolution2D(64,(3,3),activation = 'relu',name='convo4'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Flatten())
classifier.add(BatchNormalization(scale = False))
classifier.add(Dropout((0.5)))
classifier.add(Dense(1024, activation = 'relu'))
classifier.add(BatchNormalization(scale = False))
classifier.add(Dropout((0.4)))
classifier.add(Dense(10, activation = 'softmax'))


In [2]:
classifier.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

curr_path = os.getcwd()
basefolder = os.path.dirname(curr_path)
train_folder = os.path.join(basefolder, "..\DatasetITA-Ratio\Train")
test_folder = os.path.join(basefolder, "..\DatasetITA-Ratio\Test")

train_set = train_datagen.flow_from_directory(train_folder,target_size=(224, 224),batch_size=32,class_mode='categorical',color_mode='grayscale')

test_set = test_datagen.flow_from_directory(test_folder,target_size=(224, 224),batch_size=32,class_mode='categorical',color_mode='grayscale')

Found 594 images belonging to 10 classes.
Found 113 images belonging to 10 classes.


In [3]:
history = classifier.fit(train_set,steps_per_epoch=train_set.n/32,epochs=100,validation_data=test_set,validation_steps=test_set.n/32,callbacks=[csv],verbose=1)

Epoch 1/100
18/18 [==============================] - 11s 571ms/step - loss: 3.6070 - accuracy: 0.0960 - val_loss: 2.3177 - val_accuracy: 0.0973
Epoch 2/100
18/18 [==============================] - 10s 543ms/step - loss: 2.9143 - accuracy: 0.1313 - val_loss: 2.3494 - val_accuracy: 0.0885
Epoch 3/100
18/18 [==============================] - 10s 543ms/step - loss: 2.8741 - accuracy: 0.1431 - val_loss: 2.3746 - val_accuracy: 0.1062
Epoch 4/100
18/18 [==============================] - 10s 542ms/step - loss: 2.4369 - accuracy: 0.2458 - val_loss: 2.3675 - val_accuracy: 0.1062
Epoch 5/100
18/18 [==============================] - 10s 546ms/step - loss: 2.2310 - accuracy: 0.3451 - val_loss: 2.3481 - val_accuracy: 0.1327
Epoch 6/100
18/18 [==============================] - 10s 547ms/step - loss: 1.6609 - accuracy: 0.5135 - val_loss: 2.3213 - val_accuracy: 0.1150
Epoch 7/100
18/18 [==============================] - 10s 547ms/step - loss: 0.8777 - accuracy: 0.7189 - val_loss: 2.3387 - val_accuracy:

In [4]:
classifier.save('1_rmsprop_100ep.h5')